In [1]:
import os
from glob import glob
from os.path import join, split, splitext
from shutil import move

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# Split images to folders
(because of Ubuntu)

In [2]:
chunk_size = 2500
folder_path = '../data/train_unique_cropped/train_val2'
file_name_list = os.listdir(folder_path)
for start_idx in tqdm(range(0, len(file_name_list), chunk_size)):
    sub_folder_path = join(folder_path, str(start_idx))
    os.makedirs(sub_folder_path)
    for img_name in file_name_list[start_idx: start_idx + chunk_size]:
        move(join(folder_path, img_name), sub_folder_path)

  0%|          | 0/8 [00:00<?, ?it/s]

# Convert run length encoding to image labels

In [ ]:
def encoding_to_label(encoding_run_length, image_shape):
    label = np.zeros(image_shape, dtype=np.uint8)
    if type(encoding_run_length) is float:
        return label

    if type(encoding_run_length) is pd.Series:
        for encoding in encoding_run_length:
            label += encoding_to_label(encoding, image_shape)

        return label

    if type(encoding_run_length) is not list or len(encoding_run_length) % 2 == 1:
        raise Exception('invalid encoding')

    for idx in range(0, len(encoding_run_length), 2):
        start_idx = encoding_run_length[idx] - 1
        for add_idx in range(encoding_run_length[idx + 1]):
            x = (start_idx + add_idx) // image_shape[0]
            y = (start_idx + add_idx) % image_shape[0]
            label[y, x] = 255

    return label


def show(img: np.ndarray, dpi: int = 300, convert_rgb: bool = True) -> None:
    if convert_rgb:
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.figure(dpi=dpi)
    plt.imshow(img)
    plt.show()

In [ ]:
df = pd.read_csv('./data/train_ship_segmentations_v2.csv', index_col=0)
df['EncodedPixels'] = df['EncodedPixels'].map(lambda it: [int(n) for n in it.split()] if not pd.isna(it) else np.nan)
df.head()

In [ ]:
folder_path = './data/train'
for img_path in tqdm(glob(join(folder_path, '*', '*.jpg'), recursive=True)):
    if '.png' in img_path:
        continue
    img_path_base, img_name = split(img_path)
    img_name_base = splitext(img_name)[0]
    encoding = df.loc[img_name]['EncodedPixels']
    img = cv.imread(img_path)
    label = encoding_to_label(encoding, img.shape)
    cv.imwrite(join(img_path_base, f'{img_name_base}.png'), label)